<a href="https://colab.research.google.com/github/Carlo-Cascini/Carlo-Cascini/blob/main/window_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load necessary libraries
library(devtools)
library(quantmod)
library(xts)
library(zoo)
library(partialAR)
library(partialCI)
library(TTR)

getSymbols("GOOGL", src = "yahoo", from = "2020-01-01", to = "2023-12-31")
googl_prices <- Ad(GOOGL)

getSymbols("MSFT", src = "yahoo", from = "2020-01-01", to = "2023-12-31")
msft_prices <- Ad(MSFT)

prices <- merge(googl_prices, msft_prices, all = FALSE)

fit <- fit.pci(prices[, 1], prices[, 2])

print(fit)

state_history <- statehistory.pci(fit)

print(state_history)
#plot mean rev compponent
plot(index(state_history), state_history[, "M"], type = "l", ylab = "Mean-reverting component", xlab = "Time")

# Add title and labels
title(main = "Mean-reverting component", ylab = "Mean-reverting component", xlab = "Time")
# Define a list of stock tickers
stock_tickers <- c("EWA", "EWK", "EWO", "EWC", "EWQ", "EWG", "EWH",
                   "EWI", "EWJ", "EWM", "EWW", "EWN", "EWS", "EWP", "EWD", "EWL", "EWY", "EZU", "EWU", "EWZ", "EWT", "SPY",
                   "EZA", "EPI", "RSX", "TUR", "KSA", "GREK", "EIS", "ARGT", "THD", "PIN", "NORW", "IEUR", "EEM", "VWO", "AAXJ",
                   "ILF", "AFK", "FEZ", "XLF", "XLK", "XLE", "XLV", "XLY", "XLI", "XLB", "XLU", "IYR", "SMH", "XBI", "VTI", "VOO",
                   "IVV", "QQQ", "IWV", "GLD", "SLV", "PPLT", "PALL", "USO", "BNO", "UNG", "DBO", "DBC", "CORN", "SOYB", "WEAT",
                   "COW", "JJG", "UGA", "OIL", "DBA", "GSG", "COMT", "RJI", "FTGC", "BCI")

# Generate unique pairs of stock tickers
stock_pairs <- combn(stock_tickers, 2, simplify = FALSE)

# Function to calculate rsq_MR and check conditions for a given time window
calculate_rsq_MR <- function(stock_a, stock_b, start_date, end_date) {
  tryCatch({
    # Ensure stock_a is alphabetically before stock_b
    if (stock_a > stock_b) {
      temp <- stock_a
      stock_a <- stock_b
      stock_b <- temp
    }

    # Retrieve historical data for the stocks
    a_prices <- getSymbols(stock_a, src = "yahoo", from = start_date, to = end_date, auto.assign = FALSE)
    b_prices <- getSymbols(stock_b, src = "yahoo", from = start_date, to = end_date, auto.assign = FALSE)

    a_prices <- Ad(a_prices)
    b_prices <- Ad(b_prices)

    # Merge the adjusted closing prices
    prices <- merge(a_prices, b_prices, all = FALSE)
    prices_xts <- xts(prices, order.by = index(prices))

    # Fit the PCI model
    fit <- fit.pci(prices[,1], prices[,2])

    # Calculate rsq_MR
    sigmak <- fit$sigma_M
    sigmaz <- fit$sigma_R
    rho <- fit$rho
    rsq_MR <- 2 * sigmak^2 / (2 * sigmak^2 + (1 + rho) * sigmaz^2)

    # Return result including negloglik
    return(list(stock_a = stock_a, stock_b = stock_b, rsq_MR = rsq_MR, rho = rho, negloglik = fit$negloglik))
  }, error = function(e) {
    return(NULL)  # Handle errors gracefully
  })
}

# Define the starting and ending years for the analysis
start_year <- 2010
end_year <- 2016
window_length <- 5

# Initialize list to store results for trading decisions
trading_decisions <- list()

# Loop through each time window and determine trading pairs for the subsequent year
for (start in start_year:(end_year - window_length)) {
  # Define time windows for analysis and trading
  analysis_start_date <- as.Date(paste0(start, "-01-01"))
  analysis_end_date <- as.Date(paste0(start + window_length - 1, "-12-31"))
  trading_start_date <- as.Date(paste0(start + window_length, "-01-01"))
  trading_end_date <- as.Date(paste0(start + window_length + 1, "-12-31"))

  # Iterate over each stock pair and calculate the results
  results <- lapply(stock_pairs, function(pair) {
    calculate_rsq_MR(pair[1], pair[2], analysis_start_date, analysis_end_date)
  })

  # Filter out NULL results
  results <- Filter(function(x) !is.null(x), results)

  # Filter by conditions on rho and rsq_MR
  final_results <- Filter(function(x) abs(x$rho) > 0.5 && abs(x$rho) < 1 && x$rsq_MR > 0.5, results)

  # Store results for the trading period
  trading_decisions[[paste(start + window_length, start + window_length + 1, sep = "-")]] <- final_results
}

# Print results for each trading year
for (period in names(trading_decisions)) {
  cat("\nTrading Year:", period, "\n")
  if (length(trading_decisions[[period]]) > 0) {
    cat("Conditions met for the following stock pairs:\n")
    for (result in trading_decisions[[period]]) {
      cat("Stock Pair:", result$stock_a, "-", result$stock_b, "\n")
      cat("R^2[MR] =", result$rsq_MR, "\n")
      cat("rho =", result$rho, "\n")
      cat("negloglik =", result$negloglik, "\n\n")
    }
  } else {
    cat("No stock pairs met the conditions.\n")
  }
}